In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

## Preparing Data

In [2]:
with open('input.txt','r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
len(text)

1115394

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [7]:
decode(encode('i love deep learning'))

'i love deep learning'

In [8]:
data = torch.tensor(encode(text),dtype=torch.long)

In [9]:
len(data), data.dtype

(1115394, torch.int64)

In [10]:
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [12]:
xb,yb = get_batch('train')
print(xb.shape,yb.shape)
for i in range(batch_size-2):
    for j in range(block_size):
        print(xb[i,:j+1].tolist(),'-->',yb[i,j].item())

torch.Size([4, 8]) torch.Size([4, 8])
[24] --> 43
[24, 43] --> 58
[24, 43, 58] --> 5
[24, 43, 58, 5] --> 57
[24, 43, 58, 5, 57] --> 1
[24, 43, 58, 5, 57, 1] --> 46
[24, 43, 58, 5, 57, 1, 46] --> 43
[24, 43, 58, 5, 57, 1, 46, 43] --> 39
[44] --> 53
[44, 53] --> 56
[44, 53, 56] --> 1
[44, 53, 56, 1] --> 58
[44, 53, 56, 1, 58] --> 46
[44, 53, 56, 1, 58, 46] --> 39
[44, 53, 56, 1, 58, 46, 39] --> 58
[44, 53, 56, 1, 58, 46, 39, 58] --> 1


## Bigram Language Model

In [13]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,idx,targets =None):
        logits = self.token_embedding_table(idx) # B,T,C
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss
    
    def generate(self,idx,max_new_token):
        for _ in range(max_new_token):
            logits,_ = self(idx)
            logits =logits[:,-1,:] # B,C
            probs = F.softmax(logits,dim=-1)
            idx_next = torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)
        return idx
            

In [14]:
m = BigramLanguageModel(vocab_size)

In [15]:
optimizor = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [16]:
batch_size = 32
for i in range(8000):
    xb,yb = get_batch('train')
    logits,loss = m(xb,yb)
    optimizor.zero_grad(set_to_none=True)
    loss.backward()
    optimizor.step()
    if i%1000==0:
        print(loss.item())

4.648484230041504
3.7026751041412354
3.1231141090393066
2.7292535305023193
2.5700602531433105
2.491703748703003
2.4764161109924316
2.4187257289886475


In [17]:
idx = torch.zeros((1,1),dtype=torch.long)
print(decode(m.generate(idx,200)[0].tolist()))


Do  t houbar d lle ll tovan thale ermof arenore Le se my ive est irno sthes st
I be
ObRA:
KUSBins.
Pun
fouthajur,
SI'd
Agrnd thode an, ar tong-
An buirelf the wit
CKIOn w?
'l eiliow
DWe t amy tour bee


### The mathematical trick in Self-attention

In [22]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [23]:
# version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # t,C
        xbow[b,t] = torch.mean(xprev,0)
xbow.shape

torch.Size([4, 8, 32])

In [24]:
# version 2: using matrix multiply
wei = torch.tril(torch.ones(T,T))
wei= wei/wei.sum(1,keepdim=True)
xbow2 = wei @ x
xbow2.shape

torch.Size([4, 8, 32])

In [25]:
# version 3: use softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
xbow3 = wei @ x
xbow3.shape

torch.Size([4, 8, 32])

In [29]:
# version 4: use single head self-attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)

k = key(x)  # (B,T,C)@ (C,16) ---> B,T,16
q = query(x)# (B,T,C)@ (C,16) ---> B,T,16
wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) ---> (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)

v = value(x) #(B,T,C)@ (C,16) ---> B,T,16
out = wei @ v # (T,T) @ (B,T,16) ---> B,T,16
out.shape


torch.Size([4, 8, 16])